In [13]:
import mysql.connector
import json

In [14]:
path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\data\output\dump_tata_17_52.json"
with open(path, "r", encoding="utf-8") as f:
    data = json.load(f)

In [15]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1234",
    database="company_db"
)
cursor = conn.cursor()

def get_or_create_amc(amc_name):
    cursor.execute("SELECT id FROM amcs WHERE amc_name = %s", (amc_name,))
    result = cursor.fetchone()
    if result:
        return result[0]
    # cursor.execute("INSERT INTO amcs (amc_name) VALUES (%s)", (amc_name,))
    conn.commit()
    return cursor.lastrowid

In [ ]:
for scheme_name, details in data.items():
    amc_id = get_or_create_amc(details["amc_name"])
    # print(amc_id, details['amc_name'])
    
    cursor.execute("""
        INSERT INTO mutual_funds (amc_id, benchmark_index, main_scheme_name, mutual_fund_name, monthly_aaum_date, monthly_aaum_value, scheme_launch_date,min_addl_amt,min_addl_amt_multiple,min_amt,min_amt_multiple)
        VALUES (%s, %s, %s, %s, %s, %s, %s,%s,%s,%s,%s)""", 
        (amc_id, details["benchmark_index"], 
         details["main_scheme_name"], 
         details["mutual_fund_name"],
         details["monthly_aaum_date"], 
         details["monthly_aaum_value"], 
         details["scheme_launch_date"],
         details["min_addl_amt"],
         details["min_addl_amt_multiple"],
         details["min_amt"],
         details["min_amt_multiple"]))
    mutual_fund_id = cursor.lastrowid

    for manager in details.get("fund_manager", []):
        cursor.execute("""
            INSERT INTO fund_managers (main_scheme_name,mutual_fund_id, manager_name, qualification, managing_fund_since, total_exp)
            VALUES (%s, %s, %s, %s, %s, %s)""", 
            (details["main_scheme_name"],mutual_fund_id, manager["name"], manager['qualification'], manager["managing_fund_since"], manager["total_exp"]))

 
    for load_type, load_value in details.get("load", {}).items():
        cursor.execute("""
            INSERT INTO loads (main_scheme_name,mutual_fund_id, load_type, load_value)
            VALUES (%s, %s, %s, %s)""", 
            (details['main_scheme_name'],mutual_fund_id, load_type, load_value))
        
    

    for metric_name, metric_value in details.get("metrics", {}).items():
        cursor.execute("""
            INSERT INTO metrics (main_scheme_name,mutual_fund_id, name, value)
            VALUES (%s, %s, %s, %s)""", 
            (details['main_scheme_name'],mutual_fund_id, metric_name, metric_value))


conn.commit()
cursor.close()
conn.close()

print("Data inserted successfully!")

Data inserted successfully!


In [ ]:
    # min_amt = details.get("min_amt", {})
    # if min_amt:
    #     cursor.execute("""
    #         INSERT INTO fund_minimum_amounts (mutual_fund_id, amt, thraftr)
    #         VALUES (%s, %s, %s)""", 
    #         (mutual_fund_id, min_amt.get("amt", "0"), min_amt.get("thraftr", "0")))

    # min_addl_amt = details.get("min_addl_amt", {})
    # if min_addl_amt:
    #     cursor.execute("""
    #         INSERT INTO fund_additional_amounts (mutual_fund_id, amt, thraftr)
    #         VALUES (%s, %s, %s)""", 
    #         (mutual_fund_id, min_addl_amt.get("amt", "0"), min_addl_amt.get("thraftr", "0")))